In [1]:
import time
import os
import glob
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label
%matplotlib inline

# Folder paths:
test_imgs_path = 'test_images/'
output_imgs_path = 'output_images/'

In [2]:
def plot2images(img, proc_img, proc_title, output_filename, img_title='Original',
                is_img_gray=False, is_proc_gray=False):
    """
    Plots an image before and after procession.

    :param img: A given image.
    :param proc_img: The image after processing it.
    :param proc_title: Processed image's axis title.
    :param output_filename: Name of the saved file.
    :param img_title: A given image's axis title.
    :param is_img_gray: TRUE/FALSE on whether the given image is grayscaled/single channel.
    :param is_proc_gray: TRUE/FALSE on whether the proc_img is grayscaled/single channel.

    :return: NOTHING.
    """
    # # Convert from BGR (OpenCV's format) to RGB (matplotlib's format):
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # If the processed image isn't single channeled:
    # if not is_gray:
    #     proc_img = cv2.cvtColor(proc_img, cv2.COLOR_BGR2RGB)

    # Plotting - img vs proc_img:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 7))
    fig.tight_layout()

    # Given image:
    if is_img_gray:
        ax1.imshow(img, cmap='gray', interpolation='nearest')
    else:
        ax1.imshow(img, interpolation='nearest')
    ax1.set_title(img_title, fontsize=25)

    # Processed image:
    if is_proc_gray:
        ax2.imshow(proc_img, cmap='gray', interpolation='nearest')
    else:
        ax2.imshow(proc_img, interpolation='nearest')
    ax2.set_title(proc_title, fontsize=25)

    # Saving the figure:
    fig.savefig(output_imgs_path + output_filename)

#     plt.show()

In [3]:
# Creating the car and non-car datasets:
car_img_filenames = glob.glob('datasets/vehicles/**/*.png')
noncar_img_filenames = glob.glob('datasets/non-vehicles/**/*.png')
cars_len = len(car_img_filenames)
noncars_len = len(noncar_img_filenames)
print('Number of car images:', cars_len)
print('Number of non-car images:', noncars_len)

Number of car images: 8792
Number of non-car images: 8968


In [4]:
def conv_color(img, color_space='RGB', read_lib='MPL'):
    """
    Converts an image to a chosen color space.

    :param img: A given image.
    :param color_space: Chosen color space (RGB/HSV/LUV/HLS/YUV/YCrCb).
    :param read_lib: Library used for reading the image (matplotlib "MPL"/opencv "CV").

    :return: cvt_img: The converted image.
    """
    if read_lib == 'MPL':  # MatPlotLib (RGB)
        if color_space != 'RGB':
            if color_space == 'HSV':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else:
            cvt_img = np.copy(img)

    else:  # OpenCV (BGR)
        if color_space != 'RGB':
            if color_space == 'HSV':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            elif color_space == 'LUV':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
            elif color_space == 'HLS':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
            elif color_space == 'YUV':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
            elif color_space == 'YCrCb':
                cvt_img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        else:
            cvt_img = np.copy(img)

    return cvt_img

In [5]:
# Define a function to return HOG features and visualization:
def get_hog_features(img, orient, pix_per_cell, cell_per_block,
                     vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  block_norm='L2-Hys',
                                  transform_sqrt=False,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image

    # Otherwise call with one output
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       block_norm='L2-Hys',
                       transform_sqrt=False,
                       visualise=vis, feature_vector=feature_vec)
        return features

def plot_hog(img, fig_title, output_filename, color_space='RGB', orient=9, pix_per_cell=8, cell_per_block=2, vis=True, feature_vec=True):
    img = conv_color(img, color_space)

    _, hog_1st_ch = get_hog_features(img[:, :, 0], orient=orient,
                                     pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                     vis=vis, feature_vec=feature_vec)
    _, hog_2nd_ch = get_hog_features(img[:, :, 1], orient=orient,
                                     pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                     vis=vis, feature_vec=feature_vec)
    _, hog_3rd_ch = get_hog_features(img[:, :, 2], orient=orient,
                                     pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                     vis=vis, feature_vec=feature_vec)

    hog_fig, axes = plt.subplots(3, 2, figsize=(8, 10))
    hog_fig.suptitle(fig_title, fontsize=25)
    # hog_fig.tight_layout()
    axes = axes.ravel()

    axes[0].imshow(img[:, :, 0], cmap='gray', interpolation='nearest')
    axes[0].set_title(color_space[0:1] + '-CH', fontsize=10)
    axes[1].imshow(hog_1st_ch, cmap='gray', interpolation='nearest')
    axes[1].set_title('Hog ' + color_space[0:1] + '-CH', fontsize=10)

    axes[2].imshow(img[:, :, 1], cmap='gray', interpolation='nearest')
    axes[2].set_title(color_space[1:2] + '-CH', fontsize=10)
    axes[3].imshow(hog_2nd_ch, cmap='gray', interpolation='nearest')
    axes[3].set_title('Hog ' + color_space[1:2] + '-CH', fontsize=10)

    axes[4].imshow(img[:, :, 2], cmap='gray', interpolation='nearest')
    axes[4].set_title(color_space[2:3] + '-CH', fontsize=10)
    axes[5].imshow(hog_3rd_ch, cmap='gray', interpolation='nearest')
    axes[5].set_title('Hog ' + color_space[2:3] + '-CH', fontsize=10)

    # Saving the figure:
    hog_fig.savefig(output_imgs_path + output_filename)
#     plt.show()

In [6]:
# Define a function to compute binned color features
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    spat_features = cv2.resize(img, size).ravel()
    # Return the feature vector
    return spat_features

# Define a function to compute color histogram features
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:, :, 0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:, :, 1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:, :, 2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [7]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                     hist_bins=32, orient=9,
                     pix_per_cell=8, cell_per_block=2, hog_channel=0,
                     spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        feature_image = conv_color(image, color_space)

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
            # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:, :, channel],
                                                         orient, pix_per_cell, cell_per_block,
                                                         vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_image[:, :, hog_channel], orient,
                                                pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

In [8]:
if os.path.exists('model.p'):
    print('Model file found')
    with open('model.p', mode='rb') as f:
        model = pickle.load(f)
    svc = model['svc']
    X_scaler = model['scaler']
    orient = model['orient']
    pix_per_cell = model['pix_per_cell']
    cell_per_block = model['cell_per_block']
    spatial_size = model['spatial_size']
    hist_bins = model['hist_bins']
color_space = 'HSV'  # Can be RGB, HSV, LUV, HLS, YUV, YCrCb

Model file found


In [22]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    draw_img = np.copy(img)
    img = img.astype(np.float32) / 255

    img_tosearch = img[ystart:ystop, :, :]
    ctrans_tosearch = conv_color(img_tosearch, color_space='YCrCb', read_lib='CV')

    # Rescale image if not 1.0 scale:
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1] / scale), np.int(imshape[0] / scale)))

    ch1 = ctrans_tosearch[:, :, 0]
    ch2 = ctrans_tosearch[:, :, 1]
    ch3 = ctrans_tosearch[:, :, 2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1
    nfeat_per_block = orient * cell_per_block ** 2

    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1

    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)

    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb * cells_per_step
            xpos = xb * cells_per_step

            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos + nblocks_per_window, xpos:xpos + nblocks_per_window].ravel()
            hog_feat2 = hog2[ypos:ypos + nblocks_per_window, xpos:xpos + nblocks_per_window].ravel()
            hog_feat3 = hog3[ypos:ypos + nblocks_per_window, xpos:xpos + nblocks_per_window].ravel()
            hog_features = X_scaler.transform(np.hstack((hog_feat1, hog_feat2, hog_feat3)).reshape(-1, 1))
            print(len(hog_features))
            xleft = xpos * pix_per_cell
            ytop = ypos * pix_per_cell
            
#             # Extract the image patch
#             subimg = cv2.resize(ctrans_tosearch[ytop:ytop + window, xleft:xleft + window], (64, 64))

#             # Get color features
#             spatial_features = bin_spatial(subimg, size=spatial_size)
#             hist_features = color_hist(subimg, nbins=hist_bins)

#             # Scale features and make a prediction
#             test_features = X_scaler.transform(
#                 np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))
            # test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))
            test_prediction = svc.predict(hog_features.reshape(-1, 1))

            if test_prediction == 1:
                xbox_left = np.int(xleft * scale)
                ytop_draw = np.int(ytop * scale)
                win_draw = np.int(window * scale)
                cv2.rectangle(draw_img, (xbox_left, ytop_draw + ystart),
                              (xbox_left + win_draw, ytop_draw + win_draw + ystart), (0, 0, 255), 6)

    return draw_img

In [23]:
ystart = 500
ystop = 656
scale = 0.5

test_img_filename = "test6.jpg"
test_img = cv2.imread(test_imgs_path + test_img_filename)

out_img = find_cars(test_img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size,
                    hist_bins)

plt.imshow(out_img)

ValueError: non-broadcastable output operand with shape (5292,1) doesn't match the broadcast shape (5292,5388)